In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - parse_sublink_trees.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstring:
'''Parse sublink trees of MW analogs and find progenitors of high mass ratio 
mergers. Track those systems back in time and records their particles.
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np
import sys, os, pdb
import h5py
import glob
import copy
import dill as pickle

## Matplotlib
import matplotlib
from matplotlib import pyplot as plt

sys.path.insert(0,'../../src/')
from tng_dfs import util as putil
from tng_dfs import tree as ptree
from tng_dfs import plot as pplot

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [ ]:
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','RO','VO','ZO','LITTLE_H']
data_dir,ro,vo,zo,h = putil.parse_config_dict(cdict,keywords)

## API Setup & Milky Way Analogs from TNG50-1

In [ ]:
mw_mass_range = [5,7]
mwsubs = putil.prepare_mwsubs(data_dir,h=h,mw_mass_range=mw_mass_range,
    return_vars=False,force_mwsubs=False)

In [ ]:
# Setup figure directories
fig_dir = './fig/'
major_merger_types = ['mratio_at_tmax','mratio_at_tmax_ensure_mgrow',
    'mratio_at_tmax_interp_mass']
particle_types = ['dm','stars']
plot_types = ['all_merger_traces','merger_information']

for mmtype in major_merger_types:
    for ptype in particle_types:
        for plottype in plot_types:
            os.makedirs(fig_dir+'major_mergers/'+mmtype+'/'+ptype+'/'+plottype,
                exist_ok=True)

os.makedirs(fig_dir+'major_mergers/comparison/',exist_ok=True)

In [ ]:
download_trees = False
tree_dir = data_dir+'sublink_trees/full/'
if download_trees:
    for i in range(n_mw):
        putil.get(mwsubs_dict[i]['trees']['sublink'], directory=tree_dir, 
                  timeout=None )
        print('Done '+str(i))

sublink_files = np.sort(glob.glob(tree_dir+'*.hdf5'))

subhalos_select = [394621,446665,455291,477328,502371,509091,
                   517271,520885,522983,532760,537236]
subhalos_select_ind = np.where(np.isin(mwsubs['id'],subhalos_select))[0]
mwsubs = mwsubs[subhalos_select_ind]
sublink_files = sublink_files[subhalos_select_ind]

## Detect major mergers

In [ ]:
mass_ratio_threshold = 1./20.
check_descends_to_main = True

### Find major mergers using dark matter mass

In [ ]:
scheme = 'mratio_at_tmax'
scheme_kwargs = {'mass_ratio_threhsold':mass_ratio_threshold}

pz0id = []
mlpid = []
mratio = []
mratio_snap = []

for i in range(len(sublink_files)):
    # if i != 1: continue

    tree = ptree.SublinkTree(sublink_files[i])
    tree.find_where_main_branch_mass_growing(_check=True)
    tree.find_mapping_secondary_to_main_branch()
    scheme_kwargs = {} # Defaults -> mass is DM mass
    major_merger_mlpid,major_merger_mass_ratio,major_merger_mass_ratio_snap = \
        tree.find_major_mergers(scheme=scheme,
            scheme_kwargs=scheme_kwargs,
            check_descends_to_main=check_descends_to_main)
    
    # Load data for plotting
    main_leaf_progenitor_id = tree.get_property('MainLeafProgenitorID')
    subfind_id = tree.get_property('SubfindID')
    mass = tree.get_property('Mass')
    mass_star = tree.get_property('SubhaloMassType',ptype='stars')
    snap = tree.get_property('SnapNum')

    for j in range(len(major_merger_mlpid)):
        fig,axs = pplot.plot_merger_information(tree,
            mlpid=major_merger_mlpid[j], 
            snap_mass_ratio=major_merger_mass_ratio_snap[j],
            main_leaf_progenitor_id=main_leaf_progenitor_id, 
            subfind_id=subfind_id, mass=mass, mass_star=mass_star, snap=snap)
        figure_name = ('./fig/major_mergers/mratio_at_tmax/dm/'
            +'merger_information/primary_z0_SID_'
            +str(tree.main_branch_ids[0])+'_secondary_MLPID_'
            +str(major_merger_mlpid[j])+'.png')
        fig.savefig(figure_name, dpi=300)
        plt.close(fig)

    fig,axs = pplot.plot_all_merger_traces(tree,major_merger_mlpid)
    figure_name = ('./fig/major_mergers/mratio_at_tmax/dm/'
        +'all_merger_traces/primary_z0_SID_'
        +str(tree.main_branch_ids[0])+'.png')
    fig.savefig(figure_name, dpi=300)
    plt.close(fig)
    
    pz0id.append(tree.main_branch_ids[0])
    mlpid.append(major_merger_mlpid)
    mratio.append(major_merger_mass_ratio)
    mratio_snap.append(major_merger_mass_ratio_snap)

In [ ]:
for i in range(len(pz0id)):
    print('Primary z=0 SID: '+str(pz0id[i]))
    print('number of major mergers: '+str(len(mlpid[i])))
    print('major merger MLPIDs: '+str(mlpid[i]))
    print('major merger mass ratios: '+str(mratio[i]))
    print('major merger mass ratios snap nums: '+str(mratio_snap[i]))
    print('----------------------------------')

### Find major mergers using stellar mass

In [ ]:
scheme = 'mratio_at_tmax'
mass_star_dict = {'key':'SubhaloMassType','ptype':'stars'}
scheme_kwargs = {'mass_tmax_key':mass_star_dict, 
                 'mass_mratio_key':mass_star_dict,
                 'mass_ratio_threshold':mass_ratio_threshold}

pz0id_star = []
mlpid_star = []
mratio_star = []
mratio_snap_star = []

for i in range(len(sublink_files)):
    # if i != 1: continue

    tree = ptree.SublinkTree(sublink_files[i])
    tree.find_where_main_branch_mass_growing(_check=True)
    tree.find_mapping_secondary_to_main_branch()
    major_merger_mlpid,major_merger_mass_ratio,major_merger_mass_ratio_snap = \
        tree.find_major_mergers(scheme=scheme,
            scheme_kwargs=scheme_kwargs,
            check_descends_to_main=check_descends_to_main)
    
    # Load data for plotting
    main_leaf_progenitor_id = tree.get_property('MainLeafProgenitorID')
    subfind_id = tree.get_property('SubfindID')
    mass = tree.get_property('Mass')
    mass_star = tree.get_property('SubhaloMassType',ptype='stars')
    snap = tree.get_property('SnapNum')

    for j in range(len(major_merger_mlpid)):
        fig,axs = pplot.plot_merger_information(tree,
            mlpid=major_merger_mlpid[j], 
            snap_mass_star_ratio=major_merger_mass_ratio_snap[j],
            main_leaf_progenitor_id=main_leaf_progenitor_id, 
            subfind_id=subfind_id, mass=mass, mass_star=mass_star, snap=snap)
        figure_name = ('./fig/major_mergers/mratio_at_tmax/stars/'
            +'merger_information/primary_z0_SID_'
            +str(tree.main_branch_ids[0])+'_secondary_MLPID_'
            +str(major_merger_mlpid[j])+'.png')
        fig.savefig(figure_name, dpi=300)
        plt.close(fig)
    
    fig,axs = pplot.plot_all_merger_traces(tree,major_merger_mlpid)
    figure_name = ('./fig/major_mergers/mratio_at_tmax/dm/'
        +'all_merger_traces/primary_z0_SID_'
        +str(tree.main_branch_ids[0])+'.png')
    fig.savefig(figure_name, dpi=300)
    plt.close(fig)
    
    pz0id_star.append(tree.main_branch_ids[0])
    mlpid_star.append(major_merger_mlpid)
    mratio_star.append(major_merger_mass_ratio)
    mratio_snap_star.append(major_merger_mass_ratio_snap)

In [ ]:
for i in range(len(pz0id_star)):
    print('Primary z=0 SID: '+str(pz0id_star[i]))
    print('number of major mergers: '+str(len(mlpid_star[i])))
    print('major merger MLPIDs: '+str(mlpid_star[i]))
    print('major merger mass ratios: '+str(mratio[i]))
    print('major merger mass ratios snap nums: '+str(mratio_snap[i]))
    print('----------------------------------')

### Develop a plotting routine to compare two sets of major mergers

Would like a main panel comparing mass ratios determined using two different methods. Can only compare the mass ratios of secondaries which were both found in each method. Then also include secondary panels above and to the right of the main panel 

In [ ]:
scheme = 'mratio_at_tmax'
scheme_kwargs = {'mass_ratio_threshold':mass_ratio_threshold}

pz0id_1 = []
smlpid_1 = []
mratio_1 = []
mratio_snap_1 = []
for i in range(len(sublink_files)):
    tree = ptree.SublinkTree(sublink_files[i])
    tree.find_where_main_branch_mass_growing(_check=True)
    tree.find_mapping_secondary_to_main_branch()
    scheme_kwargs = {}
    major_merger_mlpid,major_merger_mass_ratio,major_merger_mass_ratio_snap\
        = tree.find_major_mergers(scheme=scheme,
            scheme_kwargs=scheme_kwargs,
            check_descends_to_main=check_descends_to_main)
    pz0id_1.append(tree.main_branch_ids[0])
    smlpid_1.append(major_merger_mlpid)
    mratio_1.append(major_merger_mass_ratio)
    mratio_snap_1.append(major_merger_mass_ratio_snap)

scheme = 'mratio_at_tmax'
mass_star_dict = {'key':'SubhaloMassType','ptype':'stars'}
scheme_kwargs = {'mass_ratio_threshold':mass_ratio_threshold,
                 'mass_tmax_key':mass_star_dict,
                 'mass_mratio_key':mass_star_dict}

pz0id_2 = []
smlpid_2 = []
mratio_2 = []
mratio_snap_2 = []
for i in range(len(sublink_files)):
    tree = ptree.SublinkTree(sublink_files[i])
    tree.find_where_main_branch_mass_growing(_check=True)
    tree.find_mapping_secondary_to_main_branch()
    mass_star_dict = {'key':'SubhaloMassType','ptype':'stars'}
    scheme_kwargs = {'mass_tmax_key':mass_star_dict,
        'mass_mratio_key':mass_star_dict}
    major_merger_mlpid,major_merger_mass_ratio,major_merger_mass_ratio_snap\
        = tree.find_major_mergers(scheme=scheme,
            scheme_kwargs=scheme_kwargs,
            check_descends_to_main=check_descends_to_main)
    pz0id_2.append(tree.main_branch_ids[0])
    smlpid_2.append(major_merger_mlpid)
    mratio_2.append(major_merger_mass_ratio)
    mratio_snap_2.append(major_merger_mass_ratio_snap)

fig,axs = pplot.plot_merger_scheme_comparison(
    [pz0id_1, smlpid_1, mratio_1, mratio_snap_1],
    [pz0id_2, smlpid_2, mratio_2, mratio_snap_2],
    plot_mlpids=True, log_mass_ratio=True
)
fig.savefig('./fig/major_mergers/comparison/mratio_at_tmax_log_comparison.png', 
    dpi=300)
plt.close(fig)

fig,axs = pplot.plot_merger_scheme_comparison(
    [pz0id_1, smlpid_1, mratio_1, mratio_snap_1],
    [pz0id_2, smlpid_2, mratio_2, mratio_snap_2],
    plot_mlpids=True, log_mass_ratio=False
)
fig.savefig('./fig/major_mergers/comparison/mratio_at_tmax_comparison.png', 
    dpi=300)
plt.close(fig)

### Now do the same comparison, detecting mergers using `mratio_at_tmax`, but also mask based on rising mass

In [ ]:
scheme = 'mratio_at_tmax'
scheme_kwargs = {'mass_ratio_threshold':mass_ratio_threshold,
                 'mask_main_branch_mass_growing':True}

pz0id_1 = []
smlpid_1 = []
mratio_1 = []
mratio_snap_1 = []
for i in range(len(sublink_files)):
    tree = ptree.SublinkTree(sublink_files[i])
    tree.find_where_main_branch_mass_growing(_check=True)
    tree.find_mapping_secondary_to_main_branch()
    major_merger_mlpid,major_merger_mass_ratio,major_merger_mass_ratio_snap\
        = tree.find_major_mergers(scheme=scheme,
            scheme_kwargs=scheme_kwargs,
            check_descends_to_main=check_descends_to_main)
    
    # Load data for plotting
    main_leaf_progenitor_id = tree.get_property('MainLeafProgenitorID')
    subfind_id = tree.get_property('SubfindID')
    mass = tree.get_property('Mass')
    mass_star = tree.get_property('SubhaloMassType',ptype='stars')
    snap = tree.get_property('SnapNum')
    
    for j in range(len(major_merger_mlpid)):
        fig,axs = pplot.plot_merger_information(tree,
            mlpid=major_merger_mlpid[j], 
            snap_mass_ratio=major_merger_mass_ratio_snap[j],
            main_leaf_progenitor_id=main_leaf_progenitor_id, 
            subfind_id=subfind_id, mass=mass, mass_star=mass_star, snap=snap)
        figure_name = ('./fig/major_mergers/mratio_at_tmax_ensure_mgrow/dm/'
            +'merger_information/primary_z0_SID_'
            +str(tree.main_branch_ids[0])+'_secondary_MLPID_'
            +str(major_merger_mlpid[j])+'.png')
        fig.savefig(figure_name, dpi=300)
        plt.close(fig)

    fig,axs = pplot.plot_all_merger_traces(tree,major_merger_mlpid)
    figure_name = ('./fig/major_mergers/mratio_at_tmax_ensure_mgrow/dm/'
        +'all_merger_traces/primary_z0_SID_'
        +str(tree.main_branch_ids[0])+'.png')
    fig.savefig(figure_name, dpi=300)
    plt.close(fig)

    pz0id_1.append(tree.main_branch_ids[0])
    smlpid_1.append(major_merger_mlpid)
    mratio_1.append(major_merger_mass_ratio)
    mratio_snap_1.append(major_merger_mass_ratio_snap)

scheme = 'mratio_at_tmax'
mass_star_dict = {'key':'SubhaloMassType','ptype':'stars'}
scheme_kwargs = {'mass_ratio_threshold':mass_ratio_threshold,
                 'mask_main_branch_mass_growing':True,
                 'mass_tmax_key':mass_star_dict,
                 'mass_mratio_key':mass_star_dict}

pz0id_2 = []
smlpid_2 = []
mratio_2 = []
mratio_snap_2 = []
for i in range(len(sublink_files)):
    tree = ptree.SublinkTree(sublink_files[i])
    tree.find_where_main_branch_mass_growing(_check=True)
    tree.find_mapping_secondary_to_main_branch()
    major_merger_mlpid,major_merger_mass_ratio,major_merger_mass_ratio_snap\
        = tree.find_major_mergers(scheme=scheme,
            scheme_kwargs=scheme_kwargs,
            check_descends_to_main=check_descends_to_main)
    
    # Load data for plotting
    main_leaf_progenitor_id = tree.get_property('MainLeafProgenitorID')
    subfind_id = tree.get_property('SubfindID')
    mass = tree.get_property('Mass')
    mass_star = tree.get_property('SubhaloMassType',ptype='stars')
    snap = tree.get_property('SnapNum')
    
    for j in range(len(major_merger_mlpid)):
        fig,axs = pplot.plot_merger_information(tree,
            mlpid=major_merger_mlpid[j], 
            snap_mass_star_ratio=major_merger_mass_ratio_snap[j],
            main_leaf_progenitor_id=main_leaf_progenitor_id, 
            subfind_id=subfind_id, mass=mass, mass_star=mass_star, snap=snap)
        figure_name = ('./fig/major_mergers/mratio_at_tmax_ensure_mgrow/stars/'
            +'merger_information/primary_z0_SID_'
            +str(tree.main_branch_ids[0])+'_secondary_MLPID_'
            +str(major_merger_mlpid[j])+'.png')
        fig.savefig(figure_name, dpi=300)
        plt.close(fig)

    fig,axs = pplot.plot_all_merger_traces(tree,major_merger_mlpid)
    figure_name = ('./fig/major_mergers/mratio_at_tmax_ensure_mgrow/stars/'
        +'all_merger_traces/primary_z0_SID_'
        +str(tree.main_branch_ids[0])+'.png')
    fig.savefig(figure_name, dpi=300)
    plt.close(fig)

    pz0id_2.append(tree.main_branch_ids[0])
    smlpid_2.append(major_merger_mlpid)
    mratio_2.append(major_merger_mass_ratio)
    mratio_snap_2.append(major_merger_mass_ratio_snap)

fig,axs = pplot.plot_merger_scheme_comparison(
    [pz0id_1, smlpid_1, mratio_1, mratio_snap_1],
    [pz0id_2, smlpid_2, mratio_2, mratio_snap_2],
    plot_mlpids=True, log_mass_ratio=False
)
fig.savefig('./fig/major_mergers/comparison/mratio_at_tmax_ensure_mgrow_comparison.png', 
    dpi=300)
plt.close(fig)

### Now do the same comparison, detect mergers using `mratio_at_tmax`, but use interpolated growing mass to determine the mass ratios 

In [ ]:
scheme = 'mratio_at_tmax'
scheme_kwargs = {'mass_ratio_threshold':mass_ratio_threshold,
                 'use_interpolated_main_branch_mass':True}

pz0id_1 = []
smlpid_1 = []
mratio_1 = []
mratio_snap_1 = []
for i in range(len(sublink_files)):
    tree = ptree.SublinkTree(sublink_files[i])
    tree.find_where_main_branch_mass_growing(_check=True)
    tree.find_mapping_secondary_to_main_branch()
    major_merger_mlpid,major_merger_mass_ratio,major_merger_mass_ratio_snap\
        = tree.find_major_mergers(scheme=scheme,
            scheme_kwargs=scheme_kwargs,
            check_descends_to_main=check_descends_to_main)
    
    # Load data for plotting
    main_leaf_progenitor_id = tree.get_property('MainLeafProgenitorID')
    subfind_id = tree.get_property('SubfindID')
    mass = tree.get_property('Mass')
    mass_star = tree.get_property('SubhaloMassType',ptype='stars')
    snap = tree.get_property('SnapNum')
    
    for j in range(len(major_merger_mlpid)):
        fig,axs = pplot.plot_merger_information(tree,
            mlpid=major_merger_mlpid[j], 
            snap_mass_ratio=major_merger_mass_ratio_snap[j],
            main_leaf_progenitor_id=main_leaf_progenitor_id, 
            subfind_id=subfind_id, mass=mass, mass_star=mass_star, snap=snap)
        figure_name = ('./fig/major_mergers/mratio_at_tmax_interp_mass/dm/'
            +'merger_information/primary_z0_SID_'
            +str(tree.main_branch_ids[0])+'_secondary_MLPID_'
            +str(major_merger_mlpid[j])+'.png')
        fig.savefig(figure_name, dpi=300)
        plt.close(fig)

    fig,axs = pplot.plot_all_merger_traces(tree,major_merger_mlpid)
    figure_name = ('./fig/major_mergers/mratio_at_tmax_interp_mass/dm/'
        +'all_merger_traces/primary_z0_SID_'
        +str(tree.main_branch_ids[0])+'.png')
    fig.savefig(figure_name, dpi=300)
    plt.close(fig)

    pz0id_1.append(tree.main_branch_ids[0])
    smlpid_1.append(major_merger_mlpid)
    mratio_1.append(major_merger_mass_ratio)
    mratio_snap_1.append(major_merger_mass_ratio_snap)

scheme = 'mratio_at_tmax'
mass_star_dict = {'key':'SubhaloMassType','ptype':'stars'}
scheme_kwargs = {'mass_ratio_threshold':mass_ratio_threshold,
                 'use_interpolated_main_branch_mass':True,
                 'mass_tmax_key':mass_star_dict,
                 'mass_mratio_key':mass_star_dict}

pz0id_2 = []
smlpid_2 = []
mratio_2 = []
mratio_snap_2 = []
for i in range(len(sublink_files)):
    tree = ptree.SublinkTree(sublink_files[i])
    tree.find_where_main_branch_mass_growing(_check=True)
    tree.find_mapping_secondary_to_main_branch()
    mass_star_dict = {'key':'SubhaloMassType','ptype':'stars'}
    scheme_kwargs = {'mass_tmax_key':mass_star_dict,
        'mass_mratio_key':mass_star_dict, 
        'use_interpolated_main_branch_mass':True}
    major_merger_mlpid,major_merger_mass_ratio,major_merger_mass_ratio_snap\
        = tree.find_major_mergers(scheme=scheme,
            scheme_kwargs=scheme_kwargs,
            check_descends_to_main=check_descends_to_main)
    
    # Load data for plotting
    main_leaf_progenitor_id = tree.get_property('MainLeafProgenitorID')
    subfind_id = tree.get_property('SubfindID')
    mass = tree.get_property('Mass')
    mass_star = tree.get_property('SubhaloMassType',ptype='stars')
    snap = tree.get_property('SnapNum')
    
    for j in range(len(major_merger_mlpid)):
        fig,axs = pplot.plot_merger_information(tree,
            mlpid=major_merger_mlpid[j], 
            snap_mass_star_ratio=major_merger_mass_ratio_snap[j],
            main_leaf_progenitor_id=main_leaf_progenitor_id, 
            subfind_id=subfind_id, mass=mass, mass_star=mass_star, snap=snap)
        figure_name = ('./fig/major_mergers/mratio_at_tmax_interp_mass/stars/'
            +'merger_information/primary_z0_SID_'
            +str(tree.main_branch_ids[0])+'_secondary_MLPID_'
            +str(major_merger_mlpid[j])+'.png')
        fig.savefig(figure_name, dpi=300)
        plt.close(fig)

    fig,axs = pplot.plot_all_merger_traces(tree,major_merger_mlpid)
    figure_name = ('./fig/major_mergers/mratio_at_tmax_interp_mass/stars/'
        +'all_merger_traces/primary_z0_SID_'
        +str(tree.main_branch_ids[0])+'.png')
    fig.savefig(figure_name, dpi=300)
    plt.close(fig)

    pz0id_2.append(tree.main_branch_ids[0])
    smlpid_2.append(major_merger_mlpid)
    mratio_2.append(major_merger_mass_ratio)
    mratio_snap_2.append(major_merger_mass_ratio_snap)

fig,axs = pplot.plot_merger_scheme_comparison(
    [pz0id_1, smlpid_1, mratio_1, mratio_snap_1],
    [pz0id_2, smlpid_2, mratio_2, mratio_snap_2],
    plot_mlpids=True, log_mass_ratio=False
)
fig.savefig('./fig/major_mergers/comparison/mratio_at_tmax_interp_mass_comparison.png', 
    dpi=300)
plt.close(fig)

### Create TreeMajorMerger and TreePrimary objects and save
The preferred method is the mass ratio at the maximum stellar mass, interpolating when the mass of the primary decreases. 

In [ ]:
scheme = 'mratio_at_tmax'
mass_star_dict = {'key':'SubhaloMassType','ptype':'stars'}
scheme_kwargs = {'mass_tmax_key':mass_star_dict, 
                 'mass_mratio_key':mass_star_dict,
                 'mass_ratio_threshold':mass_ratio_threshold,
                 'mask_main_branch_mass_growing':False,
                 'use_interpolated_main_branch_mass':True}

tree_primaries = []
tree_major_mergers = []

for i in range(len(sublink_files)):

    tree = ptree.SublinkTree(sublink_files[i])
    tree.find_where_main_branch_mass_growing(_check=True)
    tree.find_mapping_secondary_to_main_branch()
    major_merger_mlpid,major_merger_mass_ratio,major_merger_mass_ratio_snap = \
        tree.find_major_mergers(scheme=scheme,
            scheme_kwargs=scheme_kwargs,
            check_descends_to_main=check_descends_to_main)
    n_major = len(major_merger_mlpid)

    # Loop over the secondaries and create TreeMajorMerger objects
    _tree_major_mergers = []
    for j in range(n_major):

        tree_major_merger_kwargs = {
            'secondary_mlpid':major_merger_mlpid[j],
            'primary_mlpid':tree.main_branch_mlpid,
            'star_mass_ratio':major_merger_mass_ratio[j],
            'star_mass_ratio_snapnum':major_merger_mass_ratio_snap[j],
            'scheme':scheme,
            'scheme_kwargs':scheme_kwargs,
            'tree_filename':sublink_files[i],
        }
        _tree_major_mergers.append(
            ptree.TreeMajorMerger(**tree_major_merger_kwargs)
        )
    tree_major_mergers.append(_tree_major_mergers)

    # Make the primary
    tree_primary_kwargs = {
        'mlpid':tree.main_branch_mlpid,
        'tree_filename':sublink_files[i]
    }
    tree_primaries.append(
        ptree.TreePrimary(tree_major_mergers=_tree_major_mergers,
                          **tree_primary_kwargs)
    )

# Save
with open('./data/tree_primaries.pkl','wb') as f:
    pickle.dump(tree_primaries, f)
with open('./data/tree_major_mergers.pkl','wb') as f:
    pickle.dump(tree_major_mergers, f)

### Collect the information about the primaries and major mergers in dictionaries (old way of doing things)
The preferred method is the mass ratio at the maximum stellar mass, interpolating when the mass of the primary decreases. 

In [ ]:
scheme = 'mratio_at_tmax'
mass_star_dict = {'key':'SubhaloMassType','ptype':'stars'}
scheme_kwargs = {'mass_tmax_key':mass_star_dict, 
                 'mass_mratio_key':mass_star_dict,
                 'mass_ratio_threshold':mass_ratio_threshold,
                 'mask_main_branch_mass_growing':False,
                 'use_interpolated_main_branch_mass':True}
# Storage
all_major_list = []

# Loop over all sublink files
for i in range(len(sublink_files)):
    
    tree = ptree.SublinkTree(sublink_files[i])
    tree.find_where_main_branch_mass_growing(_check=True)
    tree.find_mapping_secondary_to_main_branch()
    major_merger_mlpid,major_merger_mass_ratio,major_merger_mass_ratio_snap = \
        tree.find_major_mergers(scheme=scheme,
            scheme_kwargs=scheme_kwargs,
            check_descends_to_main=check_descends_to_main)
    n_major = len(major_merger_mlpid)

    # Store the output
    primary_z0_subfind_id = tree.get_property('SubfindID')[0]
    major_list = []

    # First do the main branch
    mb_dict = {'is_primary':True,
               'mlpid':tree.get_property('MainLeafProgenitorID')[0],
               'tree_mask':tree.main_branch_mask,
               'tree_indx':np.where(tree.main_branch_mask)[0],
               'tree_ids':np.where(tree.main_branch_mask)[0],
               'snaps':tree.main_branch_snap,
               'subfind_ids':tree.get_property('SubfindID')[tree.main_branch_mask],
               }
    major_list.append(mb_dict)

    # Now do all 
    for j in range(n_major):
        this_mlpid = major_merger_mlpid[j]
        ismajor = tree.get_property('MainLeafProgenitorID') == this_mlpid
        major_dict = {'is_primary':False,
                      'mlpid':this_mlpid,
                      'tree_mask':ismajor,
                      'tree_indx':np.where(ismajor)[0],
                      'tree_ids':np.where(ismajor)[0],
                      'snaps':tree.get_property('SnapNum')[ismajor],
                      'subfind_ids':tree.get_property('SubfindID')[ismajor]
                      }
        major_list.append(major_dict)

    all_major_dict = {'primary_z0_subfind_id':primary_z0_subfind_id,
                      'n_major':n_major,
                      'major_list':major_list
                     }
    all_major_list.append(all_major_dict)

with open('./data/all_major_list.pkl','wb') as f:
    pickle.dump(all_major_list,f)